In [1]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your key: ')

Enter your key:  ········


In [4]:
test_data_path = "../data/train_dataset.xlsx"
text_data_path = "../data/hmao_npa.txt"

In [5]:
# Set text data
with open(text_data_path) as file:
    notes = file.readlines()

response = requests.post("http://localhost:8000/user-notes", json={"user_id": "2", "documents": notes})
response

<Response [200]>

In [6]:
def get_system_responses(questions):
    responses = []
    for question in tqdm.tqdm(questions.itertuples()):
        try:
            response = requests.get("http://localhost:8000/user-notes/suggestion", json={"user_id": "2", "query": question[2]})
            response.raise_for_status()
            response_data = response.json()
            responses.append(
                {
                    "question": question[2],
                    "response": response_data["llm_output"],
                    "ground_truth": question[4],
                    "context": response_data["suggestions"], 
                }
            )
        except Exception as e:
            print(f"Error while sending query: {question[1]}, error message: {e}")
            continue
    return responses

In [7]:
test_data = pd.read_excel(test_data_path)
test_data

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,1,"Каковы были основные мероприятия, реализуемые ...",['и | |развития | | |Ханты-Мансийского |исполн...,"Основные мероприятия, реализуемые Правительств...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
1,2,Каков размер финансирования из федерального бю...,['не относящиеся к районам Крайнего Севера и п...,Ответ на данный вопрос отсутствует в контексте.,simple,[{'file_path': '/home/cias/projects/other/raga...,True
2,3,Какой комплексный центр социального обслуживан...,"['|2.2. |Центр социальной |240 к/мест |7580,0 ...",Комплексный центр социального обслуживания нас...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
3,4,Какое значение имеет экологическое образование...,['(полного)общего образования и начального про...,Экологическое образование играет важную роль в...,simple,[{'file_path': '/home/cias/projects/other/raga...,True
4,6,"Каковы основные должности, соответствующие каж...",['| ||————————————————————|———————————————————...,"Основные должности, соответствующие каждому кв...",simple,[{'file_path': '/home/cias/projects/other/raga...,True
...,...,...,...,...,...,...,...
375,495,Что входит в расходы госорганов Югры?,['1 2 3 4 5 6 7 8 Дума Ханты-Мансийского автон...,Расходы государственных органов Ханты-Мансийск...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
376,496,Как изменился объем переработки газа в 2009 году?,['| | |нефтяным | | | | | | | | | |компаниям: ...,В 2009 году объем переработки газа на газопере...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True
377,497,"Что дает программа ""Развитие материально-техни...",['|3.2. |Организация и |Департамент |150 | |20...,"Программа ""Развитие материально-технической ба...",conditional,[{'file_path': '/home/cias/projects/other/raga...,True
378,498,Какую долю занимают средства Нефтеюганск?,"['¦142¦ул. Мира, дом 3 ¦ 94,50 ¦ 6483274,06 ¦0...",Средства Нефтеюганска за 2015 год составляют 2...,conditional,[{'file_path': '/home/cias/projects/other/raga...,True


In [ ]:
system_responses = get_system_responses(test_data[:32])

10it [01:11,  7.73s/it]

In [ ]:
data_samples = {
    "question": [response["question"] for response in system_responses],
    "answer": [response["response"] for response in system_responses],
    "contexts": [response["context"] for response in system_responses],
    "ground_truth": [response["ground_truth"] for response in system_responses],
}
dataset = Dataset.from_dict(data_samples)

In [ ]:
result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result